In [10]:
%matplotlib inline
# Imports
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime
from matplotlib.ticker import MultipleLocator
from scipy.stats import t as t_dist
from code.Utils import ESFitness

# Default variable definitions
num_samples = 100
available_files = [ 
    # (2, 3), 
    (5, 1),  (5, 3),  (5, 6),  (5, 9),  (5, 10),  (5, 14),  (5, 17),  (5, 20),  (5, 23),
    (10, 1), (10, 3), (10, 6), (10, 9), (10, 10), (10, 14), (10, 17), (10, 20), (10, 23),
    (20, 1), (20, 3), (20, 6), (20, 9), (20, 10), (20, 14), (20, 17), (20, 20), (20, 23),
]
# available_files = [ 
#     (5, 3),  (5, 9),  (5, 14),  (5, 17),  (5, 23),
#     (10, 3), (10, 9), (10, 14), (10, 17), (10, 23),
#     (20, 3), (20, 9), (20, 14), (20, 17), (20, 23),
# ]

folder_name = 'C:\\src\\master-thesis\\experiments\\num_runs_vs_std_dev'
data_file_name = 'raw_data\\GA_results_{ndim}dim_f{fid}.tdat'
save_file_name = 'processed_data\\samples_data_{ndim}dim_f{fid}.npz'
other_save_file_name = 'processed_data\\normalized_means_and_spread.npz'
distances_save_file = 'processed_data\\distances.npz'
plot_file_prefix = 'plots\\'

plot_file_extension = '.pdf'
fig_size = (8,6)

os.chdir(folder_name)

In [2]:
objects = {}

start = datetime.now()

# Create ESFitness objects from data files
for ndim, fid in available_files:
    if save_file_name.format(ndim=ndim, fid=fid)[15:] in os.listdir('processed_data'):
        print("{} already exists, skipping...".format(save_file_name.format(ndim=ndim, fid=fid)[15:]))
        continue
    with open(data_file_name.format(ndim=ndim, fid=fid), 'r') as f:
        lines = [line for line in f]
    objects[(ndim, fid)] = [eval(line) for line in lines]

    num_ESs = len(objects[(ndim, fid)])
    num_runs = len(objects[(ndim, fid)][0].min_fitnesses)
    
    means = np.zeros((num_runs, num_ESs, num_samples))
    medians = np.zeros((num_runs, num_ESs, num_samples))
    std_devs = np.zeros((num_runs, num_ESs, num_samples))
    
    for obj in objects[(ndim, fid)]:
        obj.min_fitnesses = np.array(obj.min_fitnesses)
    
    for sample_size in range(2, num_runs):
        samples = np.zeros((num_ESs, num_samples, sample_size))
        for sample_num in range(num_samples):
            sample_indices = np.random.choice(num_runs, sample_size, replace=False)
            for ES_num in range(num_ESs):
                obj = objects[(ndim, fid)][ES_num]
                samples[ES_num,sample_num,:] = obj.min_fitnesses[sample_indices]
    
        means[sample_size, :, :] = np.mean(samples, axis=2)
        # medians[sample_size, :, :] = np.median(samples, axis=2)
        std_devs[sample_size, :, :] = np.std(samples, axis=2)

    save_file = save_file_name.format(ndim=ndim, fid=fid)
    # np.savez(save_file, means=means, medians=medians, std_devs=std_devs)
    np.savez(save_file, means=means, std_devs=std_devs)

stop = datetime.now()
print(stop-start)

samples_data_5dim_f1.npz already exists, skipping...
samples_data_5dim_f3.npz already exists, skipping...
samples_data_5dim_f6.npz already exists, skipping...
samples_data_5dim_f9.npz already exists, skipping...
samples_data_5dim_f10.npz already exists, skipping...
samples_data_5dim_f14.npz already exists, skipping...
samples_data_5dim_f17.npz already exists, skipping...
samples_data_5dim_f20.npz already exists, skipping...
samples_data_5dim_f23.npz already exists, skipping...
samples_data_10dim_f1.npz already exists, skipping...
samples_data_10dim_f3.npz already exists, skipping...
samples_data_10dim_f6.npz already exists, skipping...
samples_data_10dim_f9.npz already exists, skipping...
samples_data_10dim_f10.npz already exists, skipping...
samples_data_10dim_f14.npz already exists, skipping...
samples_data_10dim_f17.npz already exists, skipping...
samples_data_10dim_f20.npz already exists, skipping...
samples_data_10dim_f23.npz already exists, skipping...
samples_data_20dim_f1.npz a

In [32]:
for ndim, fid in available_files:
    
    with open(save_file_name.format(ndim=ndim, fid=fid), 'rb') as save_file:
        npzfile = np.load(save_file)
        means = npzfile['means']
        num_runs, num_ESs, num_samples = means.shape
    
        ESs_to_be_plotted = range(0, num_ESs, 30)
        plot_length = num_runs//4
            
        for ES in ESs_to_be_plotted:
            x_data = range(2, plot_length)
            y_data = np.mean(means[2:plot_length,ES,:], axis=1)
            y_error = np.std(means[2:plot_length,ES,:], axis=1)
            data_mean = np.mean(y_data)
            y_data = y_data / data_mean
            y_error = y_error / data_mean
            
            plot_file_name = "mean_std_dev_errorbar_{}dim_f{}_ES{}_normalized".format(ndim, fid, ES)
            
            plt.figure(figsize=fig_size)
            plt.axhline(y=1, color='k')
            plt.errorbar(x=x_data, y=y_data, yerr=y_error, linestyle='None', marker='o')
            plt.title("Normalized Mean and Standard Deviation for ES {}/{} in {}dim F{}".format(ES, num_ESs, 
                                                                                                ndim, fid))
            plt.xlabel("Number of runs")
            plt.tight_layout()
            plt.savefig(plot_file_prefix + plot_file_name + plot_file_extension)
            plt.close()

In [11]:
all_data = []
all_errors = []
total_ESs = 0

for ndim, fid in available_files:
    
    with open(save_file_name.format(ndim=ndim, fid=fid), 'rb') as save_file:
        npzfile = np.load(save_file)
        means = npzfile['means']
        num_runs, num_ESs, num_samples = means.shape
        plot_length = num_runs
        total_ESs += num_ESs
            
        for ES in range(num_ESs):
            y_data = np.mean(means[2:plot_length,ES,:], axis=1)
            y_error = np.std(means[2:plot_length,ES,:], axis=1)
            data_mean = np.mean(y_data)
            y_data = (y_data / data_mean)
            y_error = y_error / data_mean
            
            all_data.append(y_data)
            all_errors.append(y_error)

all_data = np.mean(np.array(all_data), axis=0)
all_errors = np.mean(np.array(all_errors), axis=0)
x_data = range(2, plot_length)
np.savez(other_save_file_name, means=all_data, std_devs=all_errors, x_range=x_data)

plot_file_name = "std_err_plot_normalized_aggregated"

_, ax = plt.subplots(figsize=fig_size)
plt.grid(True, which='both', axis='y', linestyle=':', color='k', alpha=0.75)
plt.plot(x_data, all_errors, 'b-')
plt.title("Standard Error, Aggregated over {} different ESs".format(total_ESs))
plt.xlabel("Number of runs")
plt.ylabel("Relative standard error")
plt.xlim(xmax=x_data[-1])
plt.minorticks_on()
plt.tight_layout()
plt.savefig(plot_file_prefix + plot_file_name + plot_file_extension)
plt.close()

In [18]:
def plot_cdf(data, num_cases, num_bins=200, ndim=None, fid=None):
    start_index = 0
    while data[start_index] == 0:
        start_index += 1
    
    plot_data = data[start_index:]
    # Create bins so each contains an equal number of points, but remove 0-bins if many values are the same 
    bins = sorted(set(plot_data[::len(plot_data)//num_bins]))
    if plot_data[-1] > bins[-1]:
        bins.append(plot_data[-1])
    
    if ndim is not None and fid is not None:
        title = "Distance histogram for {} distances between {} ESs".format(len(data), num_cases)
        title += " in {}dim F{}".format(ndim, fid)
        plot_file_name = "distances_hist_{}dim_F{}".format(ndim, fid)
    else:
        title = "Aggregate distance histogram for {} distances between {} ESs".format(len(data), num_cases)
        plot_file_name = "distances_hist_aggregate"
    plt.figure(figsize=fig_size)
    plt.title(title)
    _, _, patches = plt.hist(plot_data, bins=bins, cumulative=True, histtype='step', normed=1)
    patches[0].set_xy(patches[0].get_xy()[:-1])  # Remove the downward line at the end 
    plt.grid(True, axis='both', linestyle=':', color='k', alpha=0.75)
    plt.xscale('log')
    plt.ylim(ymax=1)
    plt.yticks(np.arange(0, 1.1, 0.1))
    plt.tight_layout()
    plt.savefig(plot_file_prefix + plot_file_name + plot_file_extension)
    plt.close()

all_distances = []
total_ESs = 0

for ndim, fid in available_files:
    with open(data_file_name.format(ndim=ndim, fid=fid), 'r') as f:
        lines = [line for line in f]
    objects = [eval(line) for line in lines]
    FCEs = [obj.FCE for obj in objects]
    ERTs = [obj.ERT for obj in objects]
    
    num_ESs = len(FCEs)
    total_ESs += num_ESs
    distances = []
    for ES in range(num_ESs-1):
        for other_ES in range(ES+1, num_ESs):
            this = FCEs[ES]
            other = FCEs[other_ES]
            dist = np.abs(this - other)
            if dist != 0:
                distances.append(dist / min(this, other))
            else:
                this = ERTs[ES]
                other = ERTs[other_ES]
                if this is None or other is None:
                    distances.append(0)  # Apparently, distance here is actually 0... :/
                else:
                    distances.append(np.abs(this - other) / min(this, other))
    
    distances.sort()
    # plot_cdf(distances, num_ESs, ndim=ndim, fid=fid)
    
    all_distances.extend(distances)

all_distances.sort()
np.savez(distances_save_file, distances=all_distances)
print(len(all_distances))
print(all_distances[::len(all_distances)//20])

plot_cdf(all_distances, total_ESs, num_bins=1000)

271694
[0.0, 0.029405521658226862, 0.073823005157169691, 0.13007347685890025, 0.20347927289976137, 0.3005954465849387, 0.43402301913608449, 0.63528328460738781, 0.95456442504846895, 1.4722950644871693, 2.3872675169699238, 4.2920310674249098, 8.6178515443898984, 22.392458282697451, 73.941823770233682, 416.35203358269729, 5169.766250033229, 140494.2572353656, 9165357.8847258277, 457467627.84533823, 51590306299660.875]


In [19]:
distances = np.load(distances_save_file)['distances']
means = np.load(other_save_file_name)['means']
std_devs = np.load(other_save_file_name)['std_devs']
x_range = np.load(other_save_file_name)['x_range']
five_percent_points = distances[len(distances)//20::len(distances)//20]

max_num_runs = 128
probabilities = {}
mean_A = 1
for i in range(20):
    probabilities[i] = []
    mean_B = 1 + five_percent_points[i]
    
    for j, n in enumerate(x_range):
        std_dev_A = std_devs[j]
        std_dev_B = std_dev_A * mean_B
        std_error = np.sqrt((std_dev_A**2 + std_dev_B**2) / n)
        t = (mean_B - mean_A) / std_error
        
        df = 2*n - 2
        p_value = 2 * t_dist.sf(t, df=df)
        probabilities[i].append(p_value)

markers = ['bs', 'bo', 'b^', 'bv', 'b*', 
           'gs', 'go', 'g^', 'gv', 'g*', 
           'rs', 'ro', 'r^', 'rv', 'r*', 
           'ys', 'yo', 'y^', 'yv', 'y*', ]

_, ax = plt.subplots(figsize=fig_size)
ax.yaxis.set_major_locator(MultipleLocator(0.2))
ax.yaxis.set_minor_locator(MultipleLocator(0.05))
ax.xaxis.set_major_locator(MultipleLocator(20))
ax.xaxis.set_minor_locator(MultipleLocator(10))
plt.title('P-value of ES comparison at relative distance VS number of runs')
plt.xlabel('number of runs')
plt.ylabel('p-value')

for i in range(10):
    p = (i+1) * 5
    dist = five_percent_points[i]
    plt.plot(x_range, probabilities[i], markers[i], label='{}th %-ile: {:.2}'.format(p, dist))

plt.xlim(xmax=max_num_runs)
plt.minorticks_on()
plt.grid(True, axis='both', which='both', linestyle=':', color='k', alpha=0.75)    
plt.legend(numpoints=1)

plot_file_name = 'certainty_at_distance_vs_num_runs'
plt.tight_layout()
plt.savefig(plot_file_prefix + plot_file_name + plot_file_extension)
plt.close()

In [17]:
def plot_histogram(min_fitnesses, nbins=20, ndim=None, fid=None):
    if ndim is not None and fid is not None:
        plot_file_name = "fitness_hist_{}dim_F{}".format(ndim, fid)
        title = "Distribution of normalized fitness minimum values for {}dim F{}".format(ndim, fid)
    else:
        plot_file_name = "fitness_hist_aggregate"
        title = "Aggregate distribution of {} normalized fitness minimum values".format(len(min_fitnesses))

    plt.figure(figsize=fig_size)
    plt.title(title)
    plt.hist(min_fitnesses, bins=nbins, histtype='step')
    plt.yscale('log')
    plt.tight_layout()
    plt.savefig(plot_file_prefix + plot_file_name + plot_file_extension)
    plt.close()

all_FCEs = []
all_ERTs = []
all_min_fitnesses = []

for ndim, fid in available_files:
    with open(data_file_name.format(ndim=ndim, fid=fid), 'r') as f:
        min_fitnesses = []
        for line in f:
            obj = eval(line)
            min_fits = np.array(obj.min_fitnesses)
            min_fits = (min_fits / np.mean(obj.min_fitnesses)).tolist()
            min_fitnesses.extend(min_fits)

        all_min_fitnesses.extend(min_fitnesses)

plot_histogram(all_min_fitnesses, nbins=25)